In [5]:
import sys
# from cluster_manager import Clusters
import pickle
import numpy as np
import struct
import socket
import cv2
import matplotlib.pyplot as plt
import os 
import time
host = "0.0.0.0"
port = 8080

In [6]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(3)
s.bind((host, port))
s.listen(10)

def capture_frame(connection):
    data = bytearray()
    message_size = struct.calcsize("L")
    connection_valid = True

    while True:
        while len(data) < message_size:
            packet = connection.recv(4096)
            if packet == b'': 
                connection_valid = False
                break
            else: 
                data += packet
        
        if connection_valid is False:
            break
        
        data, packed_msg_size = data[message_size:], data[:message_size]
        msg_size = struct.unpack("L", packed_msg_size)[0]
        
        while len(data) < msg_size:            
            packet = connection.recv(4096)
            if packet == b'': 
                connection_valid = False
                break
            else: 
                data += packet
        
        if connection_valid is False:
            break

        data, frame_data = data[msg_size:], data[:msg_size]
        frame = pickle.loads(frame_data)
        yield frame

In [9]:
max_iter = 20
save_img = True

for iter in range(max_iter):
    try:
        connection, address = s.accept()
        print(f"Attempt {iter} of {max_iter} (Connected)")
        img_count = 1
        for frame in capture_frame(connection):
            
            cv2.imshow("img", frame)
            
            if save_img:
                with open(f"Application/iamges/frame/{img_count}", "wb") as fh:
                    pickle.dump(frame, fh, pickle.HIGHEST_PROTOCOL)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        
            img_count += 1
            
        cv2.destroyAllWindows()

    except Exception as error:
        print(f"Attempt {iter} of {max_iter} ({error})")
        time.sleep(2)

Attempt 0 of 20 (timed out)
Attempt 1 of 20 (timed out)
Attempt 2 of 20 (timed out)
Attempt 3 of 20 (timed out)
Attempt 4 of 20 (timed out)
